In [1]:
import pandas as pd

# Load data from CSV
data = pd.read_csv('C:/Users/Emumba/Documents/genie research/llm-testing-main/llm-testing-main/response validation/response-validation_llm_responses.csv')

# Ensure the data contains the necessary columns
required_columns = ["query", "ground truth", "response", "context"]
if not all(column in data.columns for column in required_columns):
    raise ValueError(f"CSV file must contain the following columns: {required_columns}")

# Mapping data columns to prompt variables
prompt_var_to_column_mapping = {
    "query": "query",
    "ground_truth": "ground truth",
    "response": "response",
    "context": "context"
}

In [2]:
accuracy_prompt = """
Question: {query}

Ground Truth: {ground_truth}

Context: {context}

Response: {response}

Evaluate the response based on Accuracy and Factuality. Check if the information provided is accurate, matches the ground truth, and does not include any unverified or fabricated details.

5: Perfectly accurate; all information directly aligns with the ground truth and context.
4: Mostly accurate, with minor inferred but plausible details.
3: Partially accurate, some minor factual inaccuracies present.
2: Noticeable inaccuracies with some correct elements.
1: Largely inaccurate or fabricated information."""

In [3]:
completeness_prompt = """
Question: {query}

Ground Truth: {ground_truth}

Context: {context}

Response: {response}

Evaluate the response based on Completeness. Does it address all aspects of the question comprehensively, covering main points and important details as per the context?

5: Fully complete; all main points and details of the question are covered.
4: Mostly complete, with minor details missing but sufficient for the main response.
3: Moderately complete, covering only parts of the question.
2: Partially complete, with several important details missing.
1: Very incomplete, missing most key points."""

In [4]:
relevance_prompt = """
Question: {query}

Ground Truth: {ground_truth}

Context: {context}

Response: {response}

Evaluate the response based on Relevance. Is the information directly related to the question, without unnecessary or unrelated details?

5: Highly relevant; every detail is directly related to the question.
4: Mostly relevant, with slight digressions.
3: Moderately relevant, with some irrelevant information.
2: Partially relevant, with considerable unrelated content.
1: Mostly irrelevant, with little focus on the question."""

In [9]:
help(CustomPromptEval)


Help on class CustomPromptEval in module uptrain.framework.evals:

class CustomPromptEval(ParametricEval)
 |  CustomPromptEval(*, prompt: str, choices: list[str], choice_scores: Union[list[float], list[int]], eval_type: Literal['classify', 'cot_classify'] = 'cot_classify', prompt_var_to_column_mapping: Optional[dict[str, str]] = None) -> None
 |  
 |  Method resolution order:
 |      CustomPromptEval
 |      ParametricEval
 |      pydantic.main.BaseModel
 |      builtins.object
 |  
 |  Data and other attributes defined here:
 |  
 |  __abstractmethods__ = frozenset()
 |  
 |  __annotations__ = {'choice_scores': typing.Union[list[float], list[int...
 |  
 |  __class_vars__ = set()
 |  
 |  __private_attributes__ = {}
 |  
 |  __pydantic_complete__ = True
 |  
 |  __pydantic_core_schema__ = {'cls': <class 'uptrain.framework.evals.Cus...
 |  
 |  __pydantic_custom_init__ = False
 |  
 |  __pydantic_decorators__ = DecoratorInfos(validators={}, field_validato...
 |  
 |  __pydantic_generic

In [ ]:
import pandas as pd
from uptrain import CustomPromptEval, EvalLLM, Settings

# Define choices and scoring for the adjusted prompt
criteria = ["5", "4", "3", "2", "1"]  # Criteria for evaluation
criteria_scores = [5, 4, 3, 2, 1]  # Score range for each criterion

# Set up evaluation with the adjusted prompt for response validation
eval_llm = EvalLLM(settings=Settings(openai_api_key="", response_format={"type": "json_object"}))



c:\Users\Emumba\AppData\Local\Programs\Python\Python311\Lib\site-packages\lazy_loader\__init__.py:202: RuntimeWarning: subpackages can technically be lazily loaded, but it causes the package to be eagerly loaded even if it is already lazily loaded.So, you probably shouldn't use subpackages with this lazy feature.
  warnings.warn(msg, RuntimeWarning)


In [10]:
# Perform individual evaluations for each criterion
# accuracy_eval = eval_llm.evaluate(
#     data=data.to_dict(orient="records"),
#     checks=[CustomPromptEval(
#         prompt=accuracy_prompt,
#         choices=criteria,
#         choice_scores=criteria_scores,
#         prompt_var_to_column_mapping=prompt_var_to_column_mapping
#     )]
# )

# completeness_eval = eval_llm.evaluate(
#     data=data.to_dict(orient="records"),
#     checks=[CustomPromptEval(
#         prompt=completeness_prompt,
#         choices=criteria,
#         choice_scores=criteria_scores,
#         prompt_var_to_column_mapping=prompt_var_to_column_mapping
#     )]
# )

relevance_eval = eval_llm.evaluate(
    data=data.to_dict(orient="records"),
    checks=[CustomPromptEval(
        prompt=relevance_prompt,
        choices=criteria,
        choice_scores=criteria_scores,
        eval_type='cot_classify',
        prompt_var_to_column_mapping=prompt_var_to_column_mapping
    )]
)
print(relevance_eval)
# Convert each result into its own DataFrame
# accuracy_df = pd.DataFrame([
#     {
#         "query": result["query"],
#         "response": result["response"],
#         "accuracy_score": result.get("score_custom_prompt", "No score")
#     }
#     for result in accuracy_eval
# ])
# completeness_df = pd.DataFrame([
#     {
#         "query": result["query"],
#         "response": result["response"],
#         "completeness_score": result.get("score_custom_prompt", "No score")
#     }
#     for result in completeness_eval
# ])
relevance_df = pd.DataFrame([
    {
        "query": result["query"],
        "response": result["response"],
        "relevance_score": result.get("score_custom_prompt", "No score")
    }
    for result in relevance_eval
])
print(relevance_df)

# Save each criterion evaluation to a separate CSV file
# accuracy_df.to_csv("accuracy_results.csv", index=False)
# completeness_df.to_csv("completeness_results.csv", index=False)
relevance_df.to_csv("relevance_results2.csv", index=False)

print("Separate CSV files created for accuracy, completeness, and relevance evaluations.")

2024-11-04 16:36:47.657 | WARNING  | uptrain.operators.language.llm:fetch_responses:268 - Detected a running event loop, scheduling requests in a separate thread.
100%|██████████| 9/9 [00:03<00:00,  2.74it/s]
c:\Users\Emumba\AppData\Local\Programs\Python\Python311\Lib\site-packages\uptrain\operators\language\llm.py:271: RuntimeWarning: coroutine 'LLMMulticlient.async_fetch_responses' was never awaited
  with ThreadPoolExecutor(max_workers=1) as executor:
2024-11-04 16:36:56.207 | INFO     | uptrain.framework.evalllm:evaluate:376 - Local server not running, start the server to log data and visualize in the dashboard!


[{'query': 'What are the embedding dimensions of GTE-Base and GTE-Large models?', 'response': 'The embedding dimensions of the GTE-Base model are 768, and those of the GTE-Large model are 1024. This information can be found in the descriptions of these models within the provided context.', 'context': "Open Source Embedding Models GTE-Base General model for similarity search or downstream enrichments. Used for general text blobs. Limited to 512 tokens Embedding Dimensions: 768 Model size: 219 MB GTE-Large High quality general model for similarity search or downstream Used for general text blobs Limited to 512 tokens Embedding Dimensions: 1024 Model Size: 670 MB GTE-Small Good quality general model for faster similarity search or downstream Used for general text blobs Limited to 512 tokens Embedding Dimensions: 384 Model Size: 67 MB E5-Small A good small and fast general model for similarity search or downstream enrichments Used for general text blobs Limited to 512 tokens Embedding Dime